In [80]:
import pandas as pd 
import numpy as np
from os import listdir
from itertools import combinations 

### Helper Functions

In [40]:
#Find number of search iterations required
#k = number of compounds that are not the protein
def number_of_search_iterations(k):
  if(k>0):
    result = k+number_of_search_iterations(k-1)
  else:
    result = 0
  return result

In [49]:
#config constraints 
tolerance = 5
protein = "Ubiquitin"

#subscript formatting for strings
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

USe Ub+C as test file

In [63]:
data_read_path = "..//Data//"
compound_list_path = data_read_path + "Compound Constraints//"
spectra_path = data_read_path + "Deconvoluted Spectra//"

#read in data
spectra_files = listdir(spectra_path)
bound_df = pd.read_excel(spectra_path+spectra_files[0])

compounds_files = listdir(compound_list_path)
compounds = pd.read_excel(compound_list_path+compounds_files[0])

In [64]:
#xtract information about Ub and the other compounds
other_compounds_df = compounds[compounds.Name != protein] #extract all compounds but Ub
protein_df = compounds[compounds.Name == protein] #extract Ub information

In [65]:
#TESTING PURPOSES

#select 3 compounds to test with
other_compounds_df = other_compounds_df.head(3)
other_compounds_df

,ID,Name,Formula,Mass
0,1,Water,H2O,19.018
1,2,Chlorine,Cl,35.976
2,3,Platinum,Pt,195.972


In [59]:
number_of_combinations = number_of_search_iterations(other_compounds_df.shape[0])
print("Number of combinations/iterations: ", number_of_combinations)

Number of combinations/iterations:  6


In [167]:
#create binding list
#binding list contains bonded compound and masses
binding_df = pd.DataFrame([])
number_of_compounds = other_compounds_df.shape[0]

for k in range(1, number_of_compounds+1):

    #how many ways can you select k items from a list of n (where n=number of compounds, k=length of combinations)
    #use ID's as from ID's we can extract the masses and compound names
    combs = combinations(other_compounds_df.ID,k)

    #create binding elements and total masses
    for bonded_compound_ids in combs:
        bonded_compound = protein_df["Formula"].values[0].translate(SUB) #Base Ub symbol
        total_mass = protein_df["Mass"].values[0] #Base Ub mass

        for idx in bonded_compound_ids:
            bonded_compound += " + " + other_compounds_df.loc[other_compounds_df.ID == idx, "Formula"].values[0].translate(SUB)
            total_mass += other_compounds_df.loc[other_compounds_df.ID == idx, "Mass"].values[0]
        binding_df = binding_df.append(pd.DataFrame({"Compound":bonded_compound, "Mass":total_mass}, index=range(1)))

binding_df.index = range(binding_df.shape[0])